# CSCI 3155 Recitation 13

April 19, 2019

### Exercise
Given the following classes:
```scala
class A {
    def m1(): String = "m1 in A"
    def m2(): String = "m2 in A"
}

abstract class B extends A {
    def m1(): String = "m1 in B"
    def m3(): String
}

trait C {
    def m2(): String
    def m3(): String = "m3 in C"
}

class D extends B with C {
    def m2(x: Int): String = "m2 in D"
    def m4(): String = "m4 in D"
}
```

Write the result of each of the following calls, then comment on how overloading, overriding, and / or inheritance result in the given behavior. Assume the following has been run before each:
```scala
val d: D = new D()
val a: A = new A()
val a_from_d: A = d
```
1. `d.m1()`
2. `d.m2()`
3. `d.m2(46)`
4. `d.m3()`
5. `d.m4()`

6. `a_from_d.m1()`
7. `a_from_d.m2()`
8. `a.m1()`
9. `a.m2()`

Next, say whether each of the following casts is valid
1. `d.asInstanceOf[A]`
2. `a_from_d.asInstanceOf[D]`
3. `a_from_d.asInstanceOf[C]`
4. `a.asInstanceOf[D]`

### Your Solution Here

## Exercise: Bringing it all together

We will create a simple class first and build to include more advanced topics each time.

### Initial attempt
The class below is our starting point

In [1]:
// Basic
import scala.collection.mutable.Queue

class Coke

class CokeVendingMachine {
    var money_collected: Int = 0

    def drop(): Coke = new Coke()
    
    def purchase(payment: Int): Coke = {
        this.money_collected += payment
        this.drop()
    }
}

import scala.collection.mutable.Queue


defined class Coke
defined class CokeVendingMachine

In [2]:
val coke_vm = new CokeVendingMachine()
coke_vm.purchase(2)

coke_vm: CokeVendingMachine = ammonite.$sess.cmd0$Helper$CokeVendingMachine@71e0f90b
res1_1: Coke = ammonite.$sess.cmd0$Helper$Coke@30a0d5ad

### Exercise: Inheritance and Generics
We want to have vending machines that dispense different items, refactor the code to use a base `VendingMachine` type with a type parameter representing the product.

In [3]:
// YOUR CODE HERE
import scala.collection.mutable.Queue

class Product
class Coke extends Product

abstract class VendingMachine[TProduct] {
    var money_collected: Int = 0
    
    def drop(): TProduct
    
    def purchase(payment: Int): TProduct = {
        this.money_collected += payment
        this.drop()
    }
}

class CokeVendingMachine extends VendingMachine[Coke] {
    def drop(): Coke = new Coke()
}

import scala.collection.mutable.Queue


defined class Product
defined class Coke
defined class VendingMachine
defined class CokeVendingMachine

In [4]:
class Chips

class ChipVendingMachine extends VendingMachine[Chips] {
    def drop() = new Chips()
}

defined class Chips
defined class ChipVendingMachine

### Exercise: Multiple Inheritance
Next, we want to add 2 abilities to our vending machines:
1. Create a `Serviceable` trait or abstract class with a method `def perform_maintenance()` and implement it for our `VendingMachine` class (so that it empties `money_collected`).
2. Create a `Shakeable` trait or abstract class with a method `def shake(): ???` and implement it for our `VendingMachine` class (so that it empties calls drop). Figure out a way to replace the `???` with the product type.

In [5]:
// YOUR CODE HERE
import scala.collection.mutable.Queue

class Coke

trait Serviceable {
    def perform_maintenance()
}

trait Shakeable[TStuffThatFallsOut] {
    def shake(): TStuffThatFallsOut
}

abstract class VendingMachine[TProduct] extends Serviceable with Shakeable[TProduct] {
    var money_collected: Int = 0
    
    def drop(): TProduct
    
    def purchase(payment: Int): TProduct = {
        this.money_collected += payment
        this.drop()
    }
    
    def perform_maintenance() {
        this.money_collected = 0
    }
    
    def shake(): TProduct = this.drop()
}

class CokeVendingMachine extends VendingMachine[Coke] {
    def drop(): Coke = new Coke()
}

import scala.collection.mutable.Queue


defined class Coke
defined trait Serviceable
defined trait Shakeable
defined class VendingMachine
defined class CokeVendingMachine

In [7]:
val coke_vm = new CokeVendingMachine()
coke_vm.purchase(2)
assert(coke_vm.money_collected == 2)
coke_vm.perform_maintenance()
assert(coke_vm.money_collected == 0)
val coke: Coke = coke_vm.shake()

class Road extends Serviceable {
    var potholes = 20
    def perform_maintenance() {
        this.potholes -= 1
    }
}

coke_vm: CokeVendingMachine = ammonite.$sess.cmd5$Helper$CokeVendingMachine@3e6a2436
res6_1: Coke = ammonite.$sess.cmd5$Helper$Coke@18b2fe7b
coke: Coke = ammonite.$sess.cmd5$Helper$Coke@2de591e3
defined class Road

### Exercise: Type Constraints
Lets introduce vending machines that take specific payment types.
Use the following to change the `purchase` method to take a specific payment type as opposed to an integer using generics:
```scala
abstract class PaymentMethod {
    def get_amount(): Int = 5 // hardcoded for example
}
abstract class Card extends PaymentMethod
class Credit extends Card
class Cash extends PaymentMethod
```
Change `VendingMachine` as needed to specify a payment type.

In [8]:
// YOUR CODE HERE
import scala.collection.mutable.Queue

class Coke

trait payment

trait Serviceable {
    def perform_maintenance()
}

trait Shakeable[TStuffThatFallsOut] {
    def shake(): TStuffThatFallsOut
}

abstract class VendingMachine[TPayment <: PaymentType, TProduct] extends Serviceable with Shakeable[TProduct] {
    var money_collected: Int = 0
    
    def drop(): TProduct
    
    def purchase(payment: TPayment): TProduct = {
        this.money_collected += payment.get_amount()
        this.drop()
    }
    
    def perform_maintenance() {
        this.money_collected = 0
    }
    
    def shake(): TProduct = this.drop()
}

class CokeVendingMachine[TPayment] extends VendingMachine[TPayment, Coke] {
    def drop(): Coke = new Coke()
}

defined trait Serviceable
defined class Coke
defined trait Shakeable
defined class PaymentMethod
defined class Card
defined class Credit
defined class Cash
defined class VendingMachine
defined class CokeVendingMachine

In [9]:
// This may not work under some versions of the scala kernel,
// but it's correct
val card_coke_vm = new CokeVendingMachine[Card]()

card_coke_vm: CokeVendingMachine[Card] = ammonite.$sess.cmd7$Helper$CokeVendingMachine@92c78bc

## Inheritance + Generics = Variance

*Reminder*: `A <: B` means `A` is a subtype / inherits from / can replace `B`.

| Name          | Scala         | English |
|---------------|---------------|---------|
| Invariant     | `class C[T]`  | If `A <: B` then `C[A]` has no relation to `C[B]`. `T` can be used anywhere.  |
| Covariant     | `class C[+T]` | If `A <: B` then `C[A] <: C[B]` (same direction). `T` can only be used as input to methods.|
| Contravariant | `class C[-T]` | If `A <: B` then `C[A] >: C[B]`  (oposite direction). `T` can only be used as output of methods. |

### Exercise: Variance
For each of the following, mark whether the class should be in- co- or contra-variant with each type parameter:

1. A `RecyclingBin` class defined as follows:
    ```scala
    class RecyclingBin[TRecycleable] {
        def recycle(trash: TRecycleable): Unit
    }
    ```

2. If we made a class `ParkingMeter[PaymentMethod]` which takes in the specified type of payment, what would it's variance be?

3. If we made a class `MovieNight[Genre]` which plays a movie of the given genre, what would it's variance be?

4. If we made a class representing functions: `Function[InType, OutType]`, should it be co- or contravariant in `InType`? `OutType`?

### Your Solution Here

### Exercise: Back to the vending machines with variance
Finally, we want the ability to use a vending machine that takes any payment type and dispenses coke in a place where a vending machine that takes cash and produces soda is expected. In other words, we want to define the variance of `VendingMachine`.

Add the proper variance to any type parameters in order to make this happen.
Use the following inheritance structure for coke:
```scala
abstract class Soda
class Coke extends Soda
```

In [10]:
// YOUR CODE HERE
???

import scala.collection.mutable.Queue


defined trait Serviceable
defined class PaymentMethod
defined class Card
defined class Credit
defined class Cash
defined class Soda
defined class Coke
defined trait Shakeable
defined class VendingMachine
defined class CokeVendingMachine

In [ ]:
// This may not work under some versions of the scala kernel,
// but it's correct
val card_coke_vm = new CokeVendingMachine[Card]()
val credit_soda_vm: VendingMachine[Credit, Soda] = card_coke_vm